In [1]:
import pyspark
sc = pyspark.SparkContext("local", "sample")

ModuleNotFoundError: No module named 'pyspark'

In [2]:
lit=sc.wholeTextFiles("lit")
ff=sc.wholeTextFiles("fanfic")

In [4]:
#removing \r characters, replace \n with a space, replacing back/forwards quotes with neutral, removing ASCII characters (currently not working)
lit=lit.map(lambda x: (x[0], x[1].replace('\r', '')))\
    .map(lambda x: (x[0], x[1].replace('\n', ' ')))\
    .map(lambda x: (x[0], x[1].replace('“', '"')))\
    .map(lambda x: (x[0], x[1].replace('”', '"')))
    #.map(lambda x: (x[0], x[1].replace('\\', '')))\
    #.map(lambda x: (x[0], x[1].replace('|', '')))\
    #.map(lambda x: (x[0], x[1].replace('/', '')))\
    #.map(lambda x: (x[0], x[1].replace('_', '')))\

ff=ff.map(lambda x: (x[0], x[1].replace('\r', '')))\
    .map(lambda x: (x[0], x[1].replace('\n', ' ')))\
    .map(lambda x: (x[0], x[1].replace('“', '"')))\
    .map(lambda x: (x[0], x[1].replace('”', '"')))
    #.map(lambda x: (x[0], x[1].replace('\\', '')))\
    #.map(lambda x: (x[0], x[1].replace('|', '')))\
    #.map(lambda x: (x[0], x[1].replace('/', '')))\
    #.map(lambda x: (x[0], x[1].replace('_', '')))\

In [5]:
# quotes pipeline for classic lit

lit2 = lit.map(lambda x: x[1])
lit_quotes1 = lit2.map(lambda x: x.split('"'))
lit_quotes2 = lit_quotes1.map(lambda x: [(i,1) for i in x])
lit_quotes3 = lit_quotes2.map(lambda x: [i[1] for i in x])

def sum_lists(x):
    sums = []
    init = 0
    for j in x:
        init = init + j
    sums.append(init)
    return sums; 

lit_quotes4 = lit_quotes3.map(sum_lists)
lit_quotes5 = lit_quotes4.flatMap(lambda x: x)
lit_quotes = lit_quotes5.map(lambda x: x - 1)
lit_quotes.collect()

[0, 176, 18, 0, 293, 0, 0, 0, 0, 0]

In [9]:
# Word count pipeline for lit

lit_words1 = lit2.map(lambda x: x.split(' '))
lit_words2 = lit_words1.map(lambda x: [(i,1) for i in x])
lit_words3 = lit_words2.map(lambda x: [i[1] for i in x])
lit_words4 = lit_words3.map(sum_lists)
lit_words = lit_words4.flatMap(lambda x: x)
lit_words.collect()

[1219, 32615, 10228, 5668, 75988, 12171, 80625, 6974, 70913, 41285]

In [11]:
# Periods pipeline for lit

lit_pds1 = lit2.map(lambda x: x.split('.'))
lit_pds2 = lit_pds1.map(lambda x: [(i,1) for i in x])
lit_pds3 = lit_pds2.map(lambda x: [i[1] for i in x])
lit_pds4 = lit_pds3.map(sum_lists)
lit_pds5 = lit_pds4.flatMap(lambda x: x)
lit_pds = lit_pds5.map(lambda x: x - 1)
lit_pds.collect()

[33, 1786, 222, 333, 2920, 805, 3903, 214, 2556, 1196]

In [17]:
# Punctuation pipeline for lit

import re
lit_puncts1 = lit2.map(lambda x: re.split(',|-|:|;', x))
lit_puncts2 = lit_puncts1.map(lambda x: [(i,1) for i in x])
lit_puncts3 = lit_puncts2.map(lambda x: [i[1] for i in x])
lit_puncts4 = lit_puncts3.map(sum_lists)
lit_puncts5 = lit_puncts4.flatMap(lambda x: x)
lit_puncts = lit_puncts5.map(lambda x: x - 1)
lit_puncts.collect()

[143, 2548, 726, 513, 6098, 1285, 6708, 436, 6649, 4192]

In [27]:
import pandas as pd

lit_df = pd.DataFrame()
lit_df['quote counts'] = lit_quotes.collect()
lit_df['prd counts'] = lit_pds.collect()
lit_df['punct counts'] = lit_puncts.collect()
lit_df['word counts'] = lit_words.collect()
lit_df['quotes/word'] = lit_df['quote counts']/lit_df['word counts']
lit_df['words/prd'] = lit_df['word counts']/lit_df['prd counts']
lit_df['puncts/prd'] = lit_df['punct counts']/lit_df['prd counts']
lit_df

,quote counts,prd counts,punct counts,word counts,quotes/word,words/prd,puncts/prd
0,0,33,143,1219,0.000000,36.939394,4.333333
1,176,1786,2548,32615,0.005396,18.261478,1.426652
2,18,222,726,10228,0.001760,46.072072,3.270270
3,0,333,513,5668,0.000000,17.021021,1.540541
4,293,2920,6098,75988,0.003856,26.023288,2.088356
5,0,805,1285,12171,0.000000,15.119255,1.596273
6,0,3903,6708,80625,0.000000,20.657187,1.718678
7,0,214,436,6974,0.000000,32.588785,2.037383
8,0,2556,6649,70913,0.000000,27.743740,2.601330
9,0,1196,4192,41285,0.000000,34.519231,3.505017


In [ ]:
# quotes pipeline for fanfic

ff2 = ff.map(lambda x: x[1])
ff_quotes1 = ff2.map(lambda x: x.split('”'))
ff_quotes2 = ff_quotes1.map(lambda x: [(i,1) for i in x])
ff_quotes3 = ff_quotes2.map(lambda x: [i[1] for i in x])

def sum_lists(x):
    sums = []
    init = 0
    for j in x:
        init = init + j
    sums.append(init)
    return sums; 

ff_quotes4 = ff_quotes3.map(sum_lists)
ff_quotes5 = ff_quotes4.flatMap(lambda x: x)
ff_quotes = ff_quotes5.map(lambda x: x - 1)
ff_quotes.collect()

[0, 0, 25, 64, 1, 0, 0, 22, 0, 25]

In [ ]:
# Word count pipeline for fanfic

ff_words1 = ff2.map(lambda x: x.split(' '))
ff_words2 = ff_words1.map(lambda x: [(i,1) for i in x])
ff_words3 = ff_words2.map(lambda x: [i[1] for i in x])
ff_words4 = ff_words3.map(sum_lists)
ff_words = ff_words4.flatMap(lambda x: x)
ff_words.collect()

[6779, 4131, 2315, 2628, 2454, 731, 7012, 3206, 758, 2820]

In [19]:
# Periods pipeline for lit

ff_pds1 = ff2.map(lambda x: x.split('.'))
ff_pds2 = ff_pds1.map(lambda x: [(i,1) for i in x])
ff_pds3 = ff_pds2.map(lambda x: [i[1] for i in x])
ff_pds4 = ff_pds3.map(sum_lists)
ff_pds5 = ff_pds4.flatMap(lambda x: x)
ff_pds = ff_pds5.map(lambda x: x - 1)
ff_pds.collect()

[506, 232, 163, 182, 162, 61, 763, 269, 46, 115]

In [20]:
# Punctuation pipeline for lit

import re
ff_puncts1 = ff2.map(lambda x: re.split(',|-|:|;', x))
ff_puncts2 = ff_puncts1.map(lambda x: [(i,1) for i in x])
ff_puncts3 = ff_puncts2.map(lambda x: [i[1] for i in x])
ff_puncts4 = ff_puncts3.map(sum_lists)
ff_puncts5 = ff_puncts4.flatMap(lambda x: x)
ff_puncts = ff_puncts5.map(lambda x: x - 1)
ff_puncts.collect()

[529, 244, 174, 167, 158, 30, 597, 191, 34, 149]

In [28]:
ff_df = pd.DataFrame()
ff_df['quote counts'] = ff_quotes.collect()
ff_df['prd counts'] = ff_pds.collect()
ff_df['punct counts'] = ff_puncts.collect()
ff_df['word counts'] = ff_words.collect()
ff_df['quotes/word'] = ff_df['quote counts']/ff_df['word counts']
ff_df['words/prd'] = ff_df['word counts']/ff_df['prd counts']
ff_df['puncts/prd'] = ff_df['punct counts']/ff_df['prd counts']
ff_df

,quote counts,prd counts,punct counts,word counts,quotes/word,words/prd,puncts/prd
0,0,506,529,6779,0.000000,13.397233,1.045455
1,0,232,244,4131,0.000000,17.806034,1.051724
2,25,163,174,2315,0.010799,14.202454,1.067485
3,64,182,167,2628,0.024353,14.439560,0.917582
4,1,162,158,2454,0.000407,15.148148,0.975309
5,0,61,30,731,0.000000,11.983607,0.491803
6,0,763,597,7012,0.000000,9.190039,0.782438
7,22,269,191,3206,0.006862,11.918216,0.710037
8,0,46,34,758,0.000000,16.478261,0.739130
9,25,115,149,2820,0.008865,24.521739,1.295652


In [31]:
ff_df['quotes/word'].mean()/lit_df['quotes/word'].mean()

4.65737052494031

In [32]:
ff_df['words/prd'].mean()/lit_df['words/prd'].mean()

0.5422358928457621

In [33]:
ff_df['puncts/prd'].mean()/lit_df['puncts/prd'].mean()

0.3763445526175359

### Development Code

In [ ]:
# erewhon1 = sc.wholeTextFiles("erewhon.txt")
# erewhon2_quotes = erewhon1.flatMap(lambda x: x[1].split('”'))

# def to_tuple(x):
#     return (x, 1)
    
# erewhon3_quotes = erewhon2_quotes.map(to_tuple)
# erewhon4_quotes = erewhon3_quotes.map(lambda x: x[1]).sum()
# erewhon_cquotes = erewhon4_quotes - 1
# erewhon_cquotes

In [ ]:
# erewhon2_periods = erewhon1.flatMap(lambda x: x[1].split('.'))
# erewhon3_periods = erewhon2_periods.map(to_tuple)
# erewhon4_periods = erewhon3_periods.map(lambda x: x[1]).sum()
# erewhon_periods = erewhon4_periods - 1

In [ ]:
# import re
# erewhon2_puncts = erewhon1.flatMap(lambda x: re.split(',|-|:|;', x[1]))
# erewhon3_puncts = erewhon2_puncts.map(to_tuple)
# erewhon4_puncts = erewhon3_puncts.map(lambda x: x[1]).sum()
# erewhon4_puncts

In [ ]:
# erewhon2_words = erewhon1.flatMap(lambda x: x[1].split(' '))
# erewhon3_words = erewhon2_words.map(to_tuple)
# erewhon4_words = erewhon3_words.map(lambda x: x[1]).sum()
# erewhon4_words

In [ ]:
# erewhon_cquotes/erewhon4_words
# erewhon4_words/erewhon_periods
# erewhon4_puncts/erewhon_periods

0.0019392372333548805